In [1]:
import torch
import numpy as np
import pandas as pd
import re
import torch
import sklearn
import nltk
import joblib
import pandas as pd

from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import set_seed

from sentence_transformers import SentenceTransformer

from sklearn.neighbors import NearestNeighbors

from utils.utils import Chunker, Generator

device = 'cuda'

set_seed(42)

nltk.download('punkt')
torch.cuda.is_available()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-03-12 22:10:06.180419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
retriever_model = SentenceTransformer("BAAI/bge-m3", device='cuda')
retriever_model.max_seq_length = 512

generator_model_name = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_model_name).cuda().eval()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
texts = pd.read_csv('documents (1).csv', header=None, names=['url','text'])
texts = texts.dropna()
texts['text'] = texts['text'].apply(lambda x: re.sub(' +', ' ', x))
texts['text'] = texts['text'].apply(lambda x: re.sub('\n+', '\n', x))
texts.url = texts.url.apply(lambda x: x if  x[-1] == '/' else x+'/')
texts.text.apply(lambda x: x.replace(x[x.find('<!--'): x.rfind('-->')+3], '').replace('Complex', '').strip('\n '))
texts = texts.set_index('url')

In [4]:
queries = pd.read_csv('Book1 (1).csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [5]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)
generator = Generator(generator_tokenizer, generator_model, max_tokens=800)

In [6]:
chunks, urls = chunker.split_texts(texts)
chunks, urls = np.array(chunks), np.array(urls)

len(chunks)

35230

In [7]:
embeddings = joblib.load('bge-m3-new-clear.pkl')
embeddings.shape

(35230, 1024)

In [8]:
# embeddings = model.encode(chunks, batch_size=32, normalize_embeddings=True, show_progress_bar=True)
# joblib.dump(embeddings, 'bge-m3-new-clear.pkl')

In [9]:
knn = NearestNeighbors(metric='cosine')
knn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [10]:
def retrieve(question, model, knn, chunks, urls):
    skores, neighbors = knn.kneighbors(model.encode(question, batch_size=32, normalize_embeddings=True).reshape(1, -1), 3)
    neighbors = neighbors.squeeze()
    retrieved_texts = list(chunks[neighbors])
    retrieved_urls = list(set(urls[neighbors]))
    return retrieved_texts, retrieved_urls

In [11]:
question = queries.question[6]
question

'Что такое ВЕБ.РФ?'

In [12]:
retrieved_texts, retrieved_urls = retrieve(question, retriever_model, knn, chunks, urls)

In [13]:
print(generator.get_answer(question, retrieved_texts, retrieved_urls, temperature=0.8))

Государственная корпорация развития "ВЭБ.РФ".

Использованные документы:
1) http://pravo.gov.ru/proxy/ips/?docbody=&nd=102114195/ 

